In [2]:
# Necessary imports
from jupyter_dash import JupyterDash
from dash import html, dash_table
import dash_leaflet as dl
from dash.dependencies import Input, Output
import pandas as pd
from AnimalShelter import AnimalShelter

# Connect to MongoDB using the AnimalShelter class
username = "aacuser"
password = "Shari1234"
try:
    shelter = AnimalShelter(username, password)
except Exception as e:
    raise Exception(f"Error initializing AnimalShelter: {e}")

# Fetch data from MongoDB and create a DataFrame
try:
    data = shelter.read({})
    df = pd.DataFrame.from_records(data)

    # Rename fields if necessary (adapt field names from your MongoDB structure)
    if 'location_lat' in df.columns and 'location_long' in df.columns:
        df.rename(columns={'location_lat': 'latitude', 'location_long': 'longitude'}, inplace=True)
    
    # Ensure required columns are present
    required_columns = ['latitude', 'longitude', 'breed', 'name']
    for col in required_columns:
        if col not in df.columns:
            df[col] = None  # Add missing columns with default values

    # Drop '_id' column if it exists
    if '_id' in df.columns:
        df.drop(columns=['_id'], inplace=True)
except Exception as e:
    raise Exception(f"Error fetching data from MongoDB: {e}")

# Initialize the app
app = JupyterDash('GeolocationDashboard')

# Dashboard layout
app.layout = html.Div([
    html.Div(id='hidden-div', style={'display': 'none'}),
    html.Center(html.B(html.H1('ShariDS SNHU CS-340 Dashboard'))),
    html.Hr(),
    # DataTable
    dash_table.DataTable(
        id='datatable-id',
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
        data=df.to_dict('records'),
        style_table={'overflowX': 'auto'},
        style_cell={
            'textAlign': 'left',
            'padding': '10px',
            'fontFamily': 'Arial',
        },
        style_header={
            'backgroundColor': 'lightblue',
            'fontWeight': 'bold'
        },
        page_size=10,
        filter_action="native",
        sort_action="native",
        row_selectable="single",
        selected_rows=[0]  # Set the first row as selected by default
    ),
    html.Br(),
    html.Hr(),
    # Geolocation chart placeholder
    html.Div(id='map-id', className='col s12 m6', style={'margin': 'auto', 'textAlign': 'center'})
])

# Callback to update the map based on the selected row
@app.callback(
    Output('map-id', 'children'),
    [Input('datatable-id', 'derived_virtual_data'),
     Input('datatable-id', 'derived_virtual_selected_rows')]
)
def update_map(viewData, index):
    # Handle empty or missing data
    if not viewData or len(viewData) == 0:
        return [
            dl.Map(style={'width': '1000px', 'height': '500px'},
                   center=[30.75, -97.48], zoom=10, children=[
                dl.TileLayer(id="base-layer-id")
            ])
        ]
    
    # Create DataFrame from the table's current state
    dff = pd.DataFrame.from_dict(viewData)

    # Ensure required columns are in the data
    required_columns = ['latitude', 'longitude', 'breed', 'name']
    if not all(col in dff.columns for col in required_columns):
        return [
            dl.Map(style={'width': '1000px', 'height': '500px'},
                   center=[30.75, -97.48], zoom=10, children=[
                dl.TileLayer(id="base-layer-id")
            ])
        ]

    # Set the default row (first row) if none is selected
    row = index[0] if index and len(index) > 0 else 0

    # Extract latitude, longitude, breed, and name
    lat = dff.iloc[row]['latitude']
    lon = dff.iloc[row]['longitude']
    breed = dff.iloc[row]['breed']
    name = dff.iloc[row]['name']

    # Return updated map with the marker
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'},
               center=[lat, lon], zoom=10, children=[
                dl.TileLayer(id="base-layer-id"),
                dl.Marker(position=[lat, lon], children=[
                    dl.Tooltip(breed),
                    dl.Popup([
                        html.H1("Animal Name"),
                        html.P(name)
                    ])
                ])
            ])
    ]

# Callback to highlight selected columns in the DataTable
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': {'column_id': i},
        'background_color': '#D2F3FF'
    } for i in selected_columns]

# Run the app
if __name__ == "__main__":
    app.run_server(mode='inline', debug=True)


---------------------------------------------------------------------------
TypeError                                 Traceback (most recent call last)
TypeError: 'NoneType' object is not iterable

